# 3. Hafta Lab-2: $L1$ ve $L2$ Regularization Sonuçlarının İncelenmesi
### SİB 552: Siber Güvenlik için Veri Madenciliği
### Gebze Teknik Üniversitesi - Siber Güvenlik Yüksek Lisans Programı
#### Dr. Ferhat Özgür Çatak
Bu lab çalışmasında lojistik regresyon algoritmasında $L2$ regularization yönteminde $\lambda$ değeriyle ağırlık vektörünün $\mathbf{w}$ değişimi KDDCUP'99 veri kümesinde uygulayacağız.

In [1]:
import numpy as np
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

In [2]:
# veri kumesini oku
kolon_adlari = ['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent',
'hot','num_failed_logins','logged_in','num_compromised','root_shell','su_attempted','num_root','num_file_creations',
'num_shells','num_access_files','num_outbound_cmds','is_host_login','is_guest_login','count','srv_count',
'serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate','srv_diff_host_rate',
'dst_host_count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate',
'dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate',
'dst_host_srv_rerror_rate','label']

verikumesi = pd.read_csv("kddcup99.tar.gz",compression="gzip", names=kolon_adlari, 
low_memory=False, skiprows=1)

# ilgili kolonlari sec
secilecek_kolonlar = ['serror_rate','srv_serror_rate',
'rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate','srv_diff_host_rate','dst_host_count',
'dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate',
'dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate',
'dst_host_srv_rerror_rate']


X = verikumesi[secilecek_kolonlar].as_matrix()
y = verikumesi['label'].apply(lambda d:0 if d == 'normal.' else 1).as_matrix()

/Users/ozgurcatak/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:21: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/ozgurcatak/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:22: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Veri kumesini egitim ve test veri kumesi olarak ayır

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

$\lambda$ değerlerini tanımla

In [4]:
lambdas = [10**-15, 10**-10, 10**-8, 0.0001, 0.001, 0.01, 1, 5, 10, 20, 50, 100, 1000]

Sonuçları kaydetmek için pandas dataframe tanımla.

In [5]:
col = ['lambda','MSE_Train','MSE_Test','w_0'] + ['w_%d'%i for i in range(1,X.shape[1]+1)] 
ind = [i for i in range(1,len(lambdas)+1)]
coef_matrix_simple = pd.DataFrame(index=ind, columns=col)
coef_matrix_simple

,lambda,MSE_Train,MSE_Test,w_0,w_1,w_2,w_3,w_4,w_5,w_6,...,w_8,w_9,w_10,w_11,w_12,w_13,w_14,w_15,w_16,w_17
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Her bir $\lambda$ değeri için $L2$ regularization kullanarak *LogisticRegression* algoritması kullanılarak sınıflandırma modelini olustur.

In [6]:
i = 0
for l in lambdas:
    clf = LogisticRegression(penalty='l2', C=1/l, solver="liblinear")
    clf.fit(X_train,y_train)
    
    y_hat_train = clf.predict(X_train)
    mse_train = mean_squared_error(y_train, y_hat_train)
    
    y_hat_test = clf.predict(X_test)
    mse_test = mean_squared_error(y_test, y_hat_test)
    
    coef_matrix_simple.iloc[i,0] = l
    coef_matrix_simple.iloc[i,1] = mse_train
    coef_matrix_simple.iloc[i,2] = mse_test
    coef_matrix_simple.iloc[i,3] = clf.intercept_[0]
    coef_matrix_simple.iloc[i,4:] = clf.coef_
    i += 1
    print('Tamamlanan : % {0}'.format(i/len(lambdas)*100), end = '\r',flush=False)
    #os.system('cls' if os.name=='nt' else 'clear')
coef_matrix_simple

,lambda,MSE_Train,MSE_Test,w_0,w_1,w_2,w_3,w_4,w_5,w_6,...,w_8,w_9,w_10,w_11,w_12,w_13,w_14,w_15,w_16,w_17
1,1e-15,0.0180004,0.0177455,-4.56687,3.17109,2.08508,-0.700451,1.55698,-7.12766,-1.19901,...,0.0319073,-0.0124543,4.2927,-6.99133,7.39695,3.87943,3.38103,2.28354,1.76187,1.36775
2,1e-10,0.0180004,0.0177455,-4.5669,3.17107,2.08508,-0.700429,1.55699,-7.12767,-1.19905,...,0.0319076,-0.0124545,4.29271,-6.99135,7.39696,3.87942,3.38102,2.28354,1.76187,1.36775
3,1e-08,0.0180004,0.0177455,-4.5669,3.17107,2.08508,-0.700431,1.55699,-7.12766,-1.19904,...,0.0319081,-0.0124551,4.29271,-6.99135,7.39695,3.87942,3.38102,2.28354,1.76187,1.36774
4,0.0001,0.0180366,0.0178375,-5.9935,4.37432,2.38566,-2.50753,2.57492,-7.37671,-2.58484,...,0.0391594,-0.0169079,5.27377,-7.58035,7.5445,7.26754,5.06514,2.951,3.10577,1.80221
5,0.001,0.0180004,0.0177455,-4.56688,3.17106,2.08506,-0.700446,1.55699,-7.12764,-1.19906,...,0.0319069,-0.0124538,4.29272,-6.99134,7.39692,3.87945,3.381,2.28352,1.76189,1.36774
6,0.01,0.018091,0.0178621,-4.57914,3.20244,2.12055,-0.696008,1.58775,-7.2947,-1.40088,...,0.0328425,-0.013061,4.37725,-7.23677,7.45113,4.02507,3.40888,2.32031,1.87204,1.40404
7,1,0.0180246,0.0177885,-4.55695,3.15686,2.06805,-0.713636,1.55673,-7.10001,-1.22202,...,0.0318837,-0.0125663,4.30116,-6.9894,7.36732,3.90755,3.36807,2.26776,1.77801,1.36663
8,5,0.0182904,0.0179786,-4.59105,3.14074,2.05097,-0.764853,1.5947,-7.148,-1.57045,...,0.0329359,-0.0136266,4.42287,-7.21555,7.30803,4.20583,3.35411,2.25908,1.99294,1.3985
9,10,0.0181605,0.0179479,-4.60079,3.03368,1.86884,-0.925433,1.60439,-6.73425,-1.42926,...,0.0311689,-0.0122815,4.21157,-6.67359,7.11849,4.19287,3.28079,2.09725,1.97594,1.37039
10,20,0.0184808,0.0182056,-4.35214,2.60879,1.69069,-0.603552,1.42058,-6.42726,-1.31222,...,0.0292089,-0.0102481,3.69843,-6.32534,6.94735,3.55271,2.7968,1.87056,1.82452,1.24571


Her bir $\lambda$ değeri için $L1$ regularization kullanarak *LogisticRegression* algoritması kullanılarak sınıflandırma modelini olustur.

In [7]:
i = 0
for l in lambdas:
    clf = LogisticRegression(penalty='l1', C=1/l, solver="liblinear")
    clf.fit(X_train,y_train)
    
    y_hat_train = clf.predict(X_train)
    mse_train = mean_squared_error(y_train, y_hat_train)
    
    y_hat_test = clf.predict(X_test)
    mse_test = mean_squared_error(y_test, y_hat_test)
    
    coef_matrix_simple.iloc[i,0] = l
    coef_matrix_simple.iloc[i,1] = mse_train
    coef_matrix_simple.iloc[i,2] = mse_test
    coef_matrix_simple.iloc[i,3] = clf.intercept_[0]
    coef_matrix_simple.iloc[i,4:] = clf.coef_
    i += 1
    print('Tamamlanan : % {0}'.format(i/len(lambdas)*100), end = '\r',flush=False)
    #os.system('cls' if os.name=='nt' else 'clear')
coef_matrix_simple

,lambda,MSE_Train,MSE_Test,w_0,w_1,w_2,w_3,w_4,w_5,w_6,...,w_8,w_9,w_10,w_11,w_12,w_13,w_14,w_15,w_16,w_17
1,1e-15,0.0159278,0.0160832,-9.25996,9.26356,-5.63814,-16.8548,13.1911,-5.67691,-1.65084,...,0.0457114,-0.0221547,6.4669,-8.06533,7.63649,14.627,6.82352,22.8792,5.65071,2.92738
2,1e-10,0.0150456,0.0152551,-8.98005,9.20676,-5.61264,-16.8346,13.1837,-5.8321,-1.77798,...,0.0452671,-0.0217674,6.35856,-8.06219,7.62655,14.472,6.80802,22.8484,5.59836,2.92618
3,1e-08,0.0150426,0.0152429,-9.1084,9.22347,-5.62324,-16.8607,13.1988,-5.76232,-1.72047,...,0.0454673,-0.0219477,6.41185,-8.0598,7.63103,14.5459,6.81272,22.8715,5.6236,2.92763
4,0.0001,0.0150487,0.0152735,-8.94753,9.18386,-5.59484,-16.846,13.1884,-5.85077,-1.79455,...,0.0452172,-0.0217451,6.35307,-8.05923,7.62483,14.4484,6.805,22.8331,5.59735,2.92693
5,0.001,0.0150577,0.0152613,-8.78555,9.15892,-5.59501,-16.8265,13.1836,-5.94639,-1.87267,...,0.0449593,-0.0215152,6.29431,-8.05124,7.62226,14.3595,6.80062,22.8286,5.55924,2.92489
6,0.01,0.0159339,0.0160894,-9.24185,9.23744,-5.61034,-16.8431,13.1782,-5.68188,-1.6549,...,0.0456683,-0.022116,6.4553,-8.0637,7.63486,14.6058,6.82157,22.843,5.64619,2.93074
7,1,0.0160185,0.0161691,-9.11943,6.77827,-2.52532,-16.4685,12.9224,-5.7591,-1.61642,...,0.0453836,-0.0219318,6.44435,-7.99076,7.61118,14.422,6.97948,19.4671,5.56661,2.89797
8,5,0.0163085,0.0164267,-8.51462,4.59928,0,-14.1438,11.2506,-6.01662,-1.6435,...,0.0439222,-0.0208139,6.21528,-7.81289,7.53339,13.5223,6.87728,14.6549,5.28177,2.45427
9,10,0.01652,0.0165433,-8.36499,4.28648,0,-11.2779,9.29489,-5.95888,-1.44611,...,0.0430324,-0.0202104,6.10961,-7.65402,7.46485,12.7153,6.58185,11.8191,5.01474,1.7791
10,20,0.0174233,0.0172855,-7.83343,4.01182,0,-6.37341,6.21552,-6.00825,-1.21594,...,0.0410427,-0.0187776,5.81562,-7.37656,7.34822,11.0344,6.07727,7.14362,4.44273,0.406635


Pandas dataframe ile **SQL** benzeri sorgular yapabilirsiniz.

In [8]:
coef_matrix_simple.query("w_2 == 0 and w_3 == 0")

,lambda,MSE_Train,MSE_Test,w_0,w_1,w_2,w_3,w_4,w_5,w_6,...,w_8,w_9,w_10,w_11,w_12,w_13,w_14,w_15,w_16,w_17
11,50,0.0182209,0.0179786,-6.37803,3.49735,0,0,1.22636,-5.89119,-0.672696,...,0.0349667,-0.014166,4.70415,-6.8741,7.12629,7.23104,5.02397,1.45465,3.07581,0
12,100,0.018946,0.0187944,-4.34296,2.07681,0,0,0.669036,-5.97376,-0.322966,...,0.0279594,-0.00890197,3.2445,-6.56496,6.93274,3.64116,3.91891,0.498042,2.98434,0
13,1000,0.0292997,0.0291056,0,0.124178,0,0,0,-6.04222,0,...,0.0154413,0.00178131,0,-3.74031,5.59417,0,0.949826,0,0,0
